In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/btckaggle/Bitcoin_kaggle.csv


In [2]:
import torch.nn as nn
import torch , torchvision
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [3]:
df = pd.read_csv("/kaggle/input/btckaggle/Bitcoin_kaggle.csv" ,  delimiter=';',
                  parse_dates=True,
                  index_col='Date' )
df

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2023-07-14,"30,346.4","29,912.7","30,442.0","29,757.4",46.42K,1.45%
2023-07-13,"30,346.4","29,912.7","30,442.0","29,757.4",46.42K,1.45%
2023-07-12,"30,346.4","29,912.7","30,442.0","29,757.4",46.42K,1.45%
2023-07-11,"30,346.4","29,912.7","30,442.0","29,757.4",46.42K,1.45%
2023-07-10,"30,346.4","29,912.7","30,442.0","29,757.4",46.42K,1.45%
...,...,...,...,...,...,...
2022-01-18,"42,364.6","42,209.9","42,674.2","41,300.7",47.32K,0.37%
2022-01-17,"42,209.3","43,080.5","43,179.6","41,559.4",41.44K,-2.02%
2022-01-16,"43,079.1","43,079.2","43,462.0","42,643.3",28.66K,-0.04%


In [4]:
df[['Price','Open','High','Low']] = df[['Price','Open','High','Low']].replace(',', '', regex=True)
df[['Price','Open','High','Low']] = df[['Price','Open','High','Low']].astype("float")

def str_to_numeric(value_str):
    suffixes = {
        'K': 10**3,
        'M': 10**6,
        'B': 10**9,
    }
    last_char = value_str[-1]
    multiplier = suffixes.get(last_char, 1)
    numeric_value = float(value_str[:-1]) * multiplier
    return numeric_value

# Applying the custom function to the 'value' column
df['Vol.'] = df['Vol.'].map(str_to_numeric)

df['Change %'] = df['Change %'].replace('%', '', regex=True)
df['Change %'] = df['Change %'].astype("float")

df


,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2023-07-14,30346.4,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-13,30346.4,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-12,30346.4,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-11,30346.4,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-10,30346.4,29912.7,30442.0,29757.4,46420.0,1.45
...,...,...,...,...,...,...
2022-01-18,42364.6,42209.9,42674.2,41300.7,47320.0,0.37
2022-01-17,42209.3,43080.5,43179.6,41559.4,41440.0,-2.02
2022-01-16,43079.1,43079.2,43462.0,42643.3,28660.0,-0.04


In [5]:
df = df.sort_index()
df

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
2022-01-14,43073.3,42562.2,43435.1,41848.7,50500.0,1.21
2022-01-15,43097.0,43073.6,43777.9,42586.1,31440.0,0.06
2022-01-16,43079.1,43079.2,43462.0,42643.3,28660.0,-0.04
2022-01-17,42209.3,43080.5,43179.6,41559.4,41440.0,-2.02
2022-01-18,42364.6,42209.9,42674.2,41300.7,47320.0,0.37
...,...,...,...,...,...,...
2023-07-10,30346.4,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-11,30346.4,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-12,30346.4,29912.7,30442.0,29757.4,46420.0,1.45


In [6]:
ind = df.index

def preprocess_data(df, lag_start=1, lag_end=20):
    
    
    target = "Price"
    df = pd.DataFrame(df.copy())
    for i in range(lag_start, lag_end):
        df[f"lag_{i}"] = df[target].shift(i)

    
    
    

    df = df.dropna()
    df = df.reset_index(drop=True)

    
    return df


In [7]:
ind

DatetimeIndex(['2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17',
               '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21',
               '2022-01-22', '2022-01-23',
               ...
               '2023-07-05', '2023-07-06', '2023-07-07', '2023-07-08',
               '2023-07-09', '2023-07-10', '2023-07-11', '2023-07-12',
               '2023-07-13', '2023-07-14'],
              dtype='datetime64[ns]', name='Date', length=547, freq=None)

In [8]:
df  = preprocess_data(df, lag_start=1, lag_end=20)

In [9]:
df.shape

(528, 25)

In [10]:
ind = ind[19:]
len(ind)

528

In [11]:
df = df.set_index(ind)
df.tail(10)

,Price,Open,High,Low,Vol.,Change %,lag_1,lag_2,lag_3,lag_4,...,lag_10,lag_11,lag_12,lag_13,lag_14,lag_15,lag_16,lag_17,lag_18,lag_19
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-05,30512.8,30768.6,30875.6,30233.3,43230.0,-0.83,30768.4,31151.3,30617.7,30586.8,...,30465.3,30533.6,30679.4,29890.5,29996.9,28307.7,26845.9,26339.7,26515.0,26341.3
2023-07-06,29913.1,30512.8,31463.6,29869.0,90810.0,-1.97,30512.8,30768.4,31151.3,30617.7,...,30267.0,30465.3,30533.6,30679.4,29890.5,29996.9,28307.7,26845.9,26339.7,26515.0
2023-07-07,30346.4,29912.7,30442.0,29757.4,46420.0,1.45,29913.1,30512.8,30768.4,31151.3,...,30689.1,30267.0,30465.3,30533.6,30679.4,29890.5,29996.9,28307.7,26845.9,26339.7
2023-07-08,30346.4,29912.7,30442.0,29757.4,46420.0,1.45,30346.4,29913.1,30512.8,30768.4,...,30078.6,30689.1,30267.0,30465.3,30533.6,30679.4,29890.5,29996.9,28307.7,26845.9
2023-07-09,30346.4,29912.7,30442.0,29757.4,46420.0,1.45,30346.4,30346.4,29913.1,30512.8,...,30445.7,30078.6,30689.1,30267.0,30465.3,30533.6,30679.4,29890.5,29996.9,28307.7
2023-07-10,30346.4,29912.7,30442.0,29757.4,46420.0,1.45,30346.4,30346.4,30346.4,29913.1,...,30472.9,30445.7,30078.6,30689.1,30267.0,30465.3,30533.6,30679.4,29890.5,29996.9
2023-07-11,30346.4,29912.7,30442.0,29757.4,46420.0,1.45,30346.4,30346.4,30346.4,30346.4,...,30586.8,30472.9,30445.7,30078.6,30689.1,30267.0,30465.3,30533.6,30679.4,29890.5
2023-07-12,30346.4,29912.7,30442.0,29757.4,46420.0,1.45,30346.4,30346.4,30346.4,30346.4,...,30617.7,30586.8,30472.9,30445.7,30078.6,30689.1,30267.0,30465.3,30533.6,30679.4
2023-07-13,30346.4,29912.7,30442.0,29757.4,46420.0,1.45,30346.4,30346.4,30346.4,30346.4,...,31151.3,30617.7,30586.8,30472.9,30445.7,30078.6,30689.1,30267.0,30465.3,30533.6


In [12]:
c = df.columns
c = list(c)
c

['Price',
 'Open',
 'High',
 'Low',
 'Vol.',
 'Change %',
 'lag_1',
 'lag_2',
 'lag_3',
 'lag_4',
 'lag_5',
 'lag_6',
 'lag_7',
 'lag_8',
 'lag_9',
 'lag_10',
 'lag_11',
 'lag_12',
 'lag_13',
 'lag_14',
 'lag_15',
 'lag_16',
 'lag_17',
 'lag_18',
 'lag_19']

In [13]:
new_cols = ['Price','lag_1','lag_2','lag_3','lag_4','lag_5','lag_6','lag_7','lag_8','lag_9','lag_10','lag_11','lag_12','lag_13','lag_14','lag_15','lag_16','lag_17','lag_18','lag_19', 'Open','High','Low','Vol.','Change %']

In [14]:
df = df[new_cols]
df.tail(10)

,Price,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,...,lag_15,lag_16,lag_17,lag_18,lag_19,Open,High,Low,Vol.,Change %
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-05,30512.8,30768.4,31151.3,30617.7,30586.8,30472.9,30445.7,30078.6,30689.1,30267.0,...,28307.7,26845.9,26339.7,26515.0,26341.3,30768.6,30875.6,30233.3,43230.0,-0.83
2023-07-06,29913.1,30512.8,30768.4,31151.3,30617.7,30586.8,30472.9,30445.7,30078.6,30689.1,...,29996.9,28307.7,26845.9,26339.7,26515.0,30512.8,31463.6,29869.0,90810.0,-1.97
2023-07-07,30346.4,29913.1,30512.8,30768.4,31151.3,30617.7,30586.8,30472.9,30445.7,30078.6,...,29890.5,29996.9,28307.7,26845.9,26339.7,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-08,30346.4,30346.4,29913.1,30512.8,30768.4,31151.3,30617.7,30586.8,30472.9,30445.7,...,30679.4,29890.5,29996.9,28307.7,26845.9,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-09,30346.4,30346.4,30346.4,29913.1,30512.8,30768.4,31151.3,30617.7,30586.8,30472.9,...,30533.6,30679.4,29890.5,29996.9,28307.7,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-10,30346.4,30346.4,30346.4,30346.4,29913.1,30512.8,30768.4,31151.3,30617.7,30586.8,...,30465.3,30533.6,30679.4,29890.5,29996.9,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-11,30346.4,30346.4,30346.4,30346.4,30346.4,29913.1,30512.8,30768.4,31151.3,30617.7,...,30267.0,30465.3,30533.6,30679.4,29890.5,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-12,30346.4,30346.4,30346.4,30346.4,30346.4,30346.4,29913.1,30512.8,30768.4,31151.3,...,30689.1,30267.0,30465.3,30533.6,30679.4,29912.7,30442.0,29757.4,46420.0,1.45
2023-07-13,30346.4,30346.4,30346.4,30346.4,30346.4,30346.4,30346.4,29913.1,30512.8,30768.4,...,30078.6,30689.1,30267.0,30465.3,30533.6,29912.7,30442.0,29757.4,46420.0,1.45


In [15]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()



df = sc.fit_transform(df)
df


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


array([[6.66628779e-01, 7.24075547e-01, 7.17410523e-01, ...,
        6.71791732e-01, 8.05819749e-06, 3.61681006e-01],
       [6.79182137e-01, 6.66628779e-01, 7.24075547e-01, ...,
        6.59579125e-01, 7.29980522e-06, 5.52945069e-01],
       [8.14307545e-01, 6.79182137e-01, 6.66628779e-01, ...,
        6.83173248e-01, 1.73982793e-05, 8.97088021e-01],
       ...,
       [4.60022480e-01, 4.60022480e-01, 4.60022480e-01, ...,
        4.52126580e-01, 6.42060711e-06, 5.65188617e-01],
       [4.60022480e-01, 4.60022480e-01, 4.60022480e-01, ...,
        4.52126580e-01, 6.42060711e-06, 5.65188617e-01],
       [4.60022480e-01, 4.60022480e-01, 4.60022480e-01, ...,
        4.52126580e-01, 6.42060711e-06, 5.65188617e-01]])

In [16]:
df = pd.DataFrame(df)
df = df.rename(columns={0 : 'Price'})
df.tail(10)

,Price,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
518,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,0.470842,0.457516,...,0.395655,0.349502,0.333520,0.339054,0.333570,0.473346,0.457362,0.467223,0.000006,0.489742
519,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,0.470842,...,0.448988,0.395655,0.349502,0.333520,0.339054,0.465269,0.475780,0.455667,0.000016,0.452019
520,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,...,0.445628,0.448988,0.395655,0.349502,0.333520,0.446322,0.443780,0.452127,0.000006,0.565189
521,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,...,0.470536,0.445628,0.448988,0.395655,0.349502,0.446322,0.443780,0.452127,0.000006,0.565189
522,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,...,0.465933,0.470536,0.445628,0.448988,0.395655,0.446322,0.443780,0.452127,0.000006,0.565189
523,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,...,0.463776,0.465933,0.470536,0.445628,0.448988,0.446322,0.443780,0.452127,0.000006,0.565189
524,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,...,0.457516,0.463776,0.465933,0.470536,0.445628,0.446322,0.443780,0.452127,0.000006,0.565189
525,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,...,0.470842,0.457516,0.463776,0.465933,0.470536,0.446322,0.443780,0.452127,0.000006,0.565189
526,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,...,0.451567,0.470842,0.457516,0.463776,0.465933,0.446322,0.443780,0.452127,0.000006,0.565189
527,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,...,0.463158,0.451567,0.470842,0.457516,0.463776,0.446322,0.443780,0.452127,0.000006,0.565189


In [17]:
df.head(10)

,Price,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.666629,0.724076,0.717411,0.699070,0.707061,0.693620,0.675283,0.663794,0.668536,0.660191,...,0.839471,0.834568,0.862030,0.862595,0.861847,0.724119,0.707271,0.671792,0.000008,0.361681
1,0.679182,0.666629,0.724076,0.717411,0.699070,0.707061,0.693620,0.675283,0.663794,0.668536,...,0.817787,0.839471,0.834568,0.862030,0.862595,0.666625,0.658734,0.659579,0.000007,0.552945
2,0.814308,0.679182,0.666629,0.724076,0.717411,0.699070,0.707061,0.693620,0.675283,0.663794,...,0.787417,0.817787,0.839471,0.834568,0.862030,0.679178,0.796081,0.683173,0.000017,0.897088
3,0.809398,0.814308,0.679182,0.666629,0.724076,0.717411,0.699070,0.707061,0.693620,0.675283,...,0.653536,0.787417,0.817787,0.839471,0.834568,0.814270,0.802684,0.807590,0.000006,0.504964
4,0.840343,0.809398,0.814308,0.679182,0.666629,0.724076,0.717411,0.699070,0.707061,0.693620,...,0.609324,0.653536,0.787417,0.817787,0.839471,0.809389,0.825081,0.814346,0.000003,0.595632
5,0.885861,0.840343,0.809398,0.814308,0.679182,0.666629,0.724076,0.717411,0.699070,0.707061,...,0.647032,0.609324,0.653536,0.787417,0.817787,0.840119,0.883140,0.830813,0.000013,0.629715
6,0.892311,0.885861,0.840343,0.809398,0.814308,0.679182,0.666629,0.724076,0.717411,0.699070,...,0.660191,0.647032,0.609324,0.653536,0.787417,0.885860,0.912318,0.862838,0.000017,0.532760
7,0.903384,0.892311,0.885861,0.840343,0.809398,0.814308,0.679182,0.666629,0.724076,0.717411,...,0.668536,0.660191,0.647032,0.609324,0.653536,0.892310,0.893542,0.877366,0.000007,0.543680
8,0.875982,0.903384,0.892311,0.885861,0.840343,0.809398,0.814308,0.679182,0.666629,0.724076,...,0.663794,0.668536,0.660191,0.647032,0.609324,0.903383,0.923450,0.879888,0.000016,0.452349
9,0.840223,0.875982,0.903384,0.892311,0.885861,0.840343,0.809398,0.814308,0.679182,0.666629,...,0.675283,0.663794,0.668536,0.660191,0.647032,0.875927,0.865811,0.841217,0.000012,0.431171


In [18]:
df.tail(10)

,Price,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
518,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,0.470842,0.457516,...,0.395655,0.349502,0.333520,0.339054,0.333570,0.473346,0.457362,0.467223,0.000006,0.489742
519,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,0.470842,...,0.448988,0.395655,0.349502,0.333520,0.339054,0.465269,0.475780,0.455667,0.000016,0.452019
520,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,...,0.445628,0.448988,0.395655,0.349502,0.333520,0.446322,0.443780,0.452127,0.000006,0.565189
521,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,...,0.470536,0.445628,0.448988,0.395655,0.349502,0.446322,0.443780,0.452127,0.000006,0.565189
522,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,...,0.465933,0.470536,0.445628,0.448988,0.395655,0.446322,0.443780,0.452127,0.000006,0.565189
523,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,...,0.463776,0.465933,0.470536,0.445628,0.448988,0.446322,0.443780,0.452127,0.000006,0.565189
524,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,...,0.457516,0.463776,0.465933,0.470536,0.445628,0.446322,0.443780,0.452127,0.000006,0.565189
525,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,...,0.470842,0.457516,0.463776,0.465933,0.470536,0.446322,0.443780,0.452127,0.000006,0.565189
526,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,...,0.451567,0.470842,0.457516,0.463776,0.465933,0.446322,0.443780,0.452127,0.000006,0.565189
527,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,...,0.463158,0.451567,0.470842,0.457516,0.463776,0.446322,0.443780,0.452127,0.000006,0.565189


In [19]:
# 520 521))) 522 523 524 525 526 527 528 
y_train = df.loc[ 0 : 520 ,'Price']
y_test = df.loc[521 : , 'Price' ]

In [20]:
y_test

521    0.460022
522    0.460022
523    0.460022
524    0.460022
525    0.460022
526    0.460022
527    0.460022
Name: Price, dtype: float64

In [21]:
df = df.drop(columns = ['Price'])
df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,0.724076,0.717411,0.699070,0.707061,0.693620,0.675283,0.663794,0.668536,0.660191,0.647032,...,0.839471,0.834568,0.862030,0.862595,0.861847,0.724119,0.707271,0.671792,0.000008,0.361681
1,0.666629,0.724076,0.717411,0.699070,0.707061,0.693620,0.675283,0.663794,0.668536,0.660191,...,0.817787,0.839471,0.834568,0.862030,0.862595,0.666625,0.658734,0.659579,0.000007,0.552945
2,0.679182,0.666629,0.724076,0.717411,0.699070,0.707061,0.693620,0.675283,0.663794,0.668536,...,0.787417,0.817787,0.839471,0.834568,0.862030,0.679178,0.796081,0.683173,0.000017,0.897088
3,0.814308,0.679182,0.666629,0.724076,0.717411,0.699070,0.707061,0.693620,0.675283,0.663794,...,0.653536,0.787417,0.817787,0.839471,0.834568,0.814270,0.802684,0.807590,0.000006,0.504964
4,0.809398,0.814308,0.679182,0.666629,0.724076,0.717411,0.699070,0.707061,0.693620,0.675283,...,0.609324,0.653536,0.787417,0.817787,0.839471,0.809389,0.825081,0.814346,0.000003,0.595632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,...,0.463776,0.465933,0.470536,0.445628,0.448988,0.446322,0.443780,0.452127,0.000006,0.565189
524,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,...,0.457516,0.463776,0.465933,0.470536,0.445628,0.446322,0.443780,0.452127,0.000006,0.565189
525,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,...,0.470842,0.457516,0.463776,0.465933,0.470536,0.446322,0.443780,0.452127,0.000006,0.565189
526,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,...,0.451567,0.470842,0.457516,0.463776,0.465933,0.446322,0.443780,0.452127,0.000006,0.565189


In [22]:
df.tail(10)

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
518,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,0.470842,0.457516,0.463776,...,0.395655,0.349502,0.333520,0.339054,0.333570,0.473346,0.457362,0.467223,0.000006,0.489742
519,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,0.470842,0.457516,...,0.448988,0.395655,0.349502,0.333520,0.339054,0.465269,0.475780,0.455667,0.000016,0.452019
520,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,0.470842,...,0.445628,0.448988,0.395655,0.349502,0.333520,0.446322,0.443780,0.452127,0.000006,0.565189
521,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,0.451567,...,0.470536,0.445628,0.448988,0.395655,0.349502,0.446322,0.443780,0.452127,0.000006,0.565189
522,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,0.463158,...,0.465933,0.470536,0.445628,0.448988,0.395655,0.446322,0.443780,0.452127,0.000006,0.565189
523,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,0.464016,...,0.463776,0.465933,0.470536,0.445628,0.448988,0.446322,0.443780,0.452127,0.000006,0.565189
524,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,0.467613,...,0.457516,0.463776,0.465933,0.470536,0.445628,0.446322,0.443780,0.452127,0.000006,0.565189
525,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,0.468588,...,0.470842,0.457516,0.463776,0.465933,0.470536,0.446322,0.443780,0.452127,0.000006,0.565189
526,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,0.485435,...,0.451567,0.470842,0.457516,0.463776,0.465933,0.446322,0.443780,0.452127,0.000006,0.565189
527,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.460022,0.446342,0.465276,0.473346,...,0.463158,0.451567,0.470842,0.457516,0.463776,0.446322,0.443780,0.452127,0.000006,0.565189


In [23]:
X_test = df.loc[521 : , :]
X_train = df.loc[0 : 520 , : ]

X_test.shape , X_train.shape

((7, 24), (521, 24))

In [24]:
X_train_tensor = X_train.values
y_train_tensor = y_train.values
X_test_tensor = X_test.values
y_test_tensor = y_test.values

y_train_tensor_total = []
for i in y_train_tensor :
    
    temp = []
    
    i = temp.append(i)
    y_train_tensor_total.append(temp)
    
y_train_tensor =   y_train_tensor_total


y_test_tensor_total = list()
for i in y_test_tensor :
    
    temp = []
    
    i = temp.append(i)
    y_test_tensor_total.append(temp)
    
y_test_tensor =   y_test_tensor_total

In [25]:
X_train_tensor = torch.tensor(X_train_tensor).float()
y_train_tensor = torch.tensor(y_train_tensor).float()
X_test_tensor = torch.tensor(X_test_tensor).float()
y_test_tensor = torch.tensor(y_test_tensor).float()

X_train_tensor.shape ,y_train_tensor.shape , X_test_tensor.shape , y_test_tensor.shape




(torch.Size([521, 24]),
 torch.Size([521, 1]),
 torch.Size([7, 24]),
 torch.Size([7, 1]))

In [26]:

X_train_tensor.shape

torch.Size([521, 24])

In [27]:
X_train_tensor.size(0)

521

In [28]:
y_train_tensor

tensor([[0.6666],
        [0.6792],
        [0.8143],
        [0.8094],
        [0.8403],
        [0.8859],
        [0.8923],
        [0.9034],
        [0.8760],
        [0.8402],
        [0.8344],
        [0.8299],
        [0.8453],
        [0.9083],
        [0.8874],
        [0.7823],
        [0.7648],
        [0.7677],
        [0.7129],
        [0.6707],
        [0.7095],
        [0.6772],
        [0.7124],
        [0.7399],
        [0.7369],
        [0.6919],
        [0.8655],
        [0.9044],
        [0.8884],
        [0.8426],
        [0.7377],
        [0.7457],
        [0.7144],
        [0.7025],
        [0.7252],
        [0.8257],
        [0.7464],
        [0.7247],
        [0.7274],
        [0.6951],
        [0.7544],
        [0.7423],
        [0.8001],
        [0.7937],
        [0.8206],
        [0.8353],
        [0.8051],
        [0.7971],
        [0.8397],
        [0.8568],
        [0.8915],
        [0.9016],
        [0.9084],
        [0.9814],
        [0.9891],
        [1

In [29]:
# creating train dataset
train_dataset = []

for i in range(X_train_tensor.size(0)) :
    temp_tuple =  X_train_tensor[i] , y_train_tensor[i]
    train_dataset.append(temp_tuple)
    
    
    



In [30]:
from torch.utils.data import DataLoader

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle= False)

In [31]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    

torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 24]) torch.Size([16, 1])
torch.Size([16, 

In [32]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x

# Гиперпараметры
input_size = 24
hidden_size = 12
output_size = 1

# Создание экземпляра модели
model = SimpleNN(input_size, hidden_size, output_size)

model.to(device)
print(model)


SimpleNN(
  (fc1): Linear(in_features=24, out_features=12, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=12, out_features=1, bias=True)
)


In [33]:
from torchmetrics.regression import MeanAbsoluteError
import torch.nn.functional as F


learning_rate = 0.003
num_epochs = 25
loss_function = nn.L1Loss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [34]:
def train_one_epoch():
    
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(x_batch)
        
        loss = loss_function(output, y_batch)
        running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    l = running_loss
    print(l)

In [35]:
for epoch in range(num_epochs):
    train_one_epoch()

Epoch: 1
4.723127633333206
Epoch: 2
2.498740764334798
Epoch: 3
2.5436353255063295
Epoch: 4
2.332989877089858
Epoch: 5
2.191053302027285
Epoch: 6
2.240351280197501
Epoch: 7
2.113879157230258
Epoch: 8
2.071982898749411
Epoch: 9
2.0245296340435743
Epoch: 10
1.9578183586709201
Epoch: 11
2.001828281208873
Epoch: 12
1.8049911363050342
Epoch: 13
1.9657580619677901
Epoch: 14
1.8854931322857738
Epoch: 15
1.6113916779868305
Epoch: 16
1.9754376010969281
Epoch: 17
1.846407930366695
Epoch: 18
1.1091005373746157
Epoch: 19
1.7384667545557022
Epoch: 20
1.9155606674030423
Epoch: 21
0.9446104690432549
Epoch: 22
1.4370463490486145
Epoch: 23
1.8717831876128912
Epoch: 24
0.9755871323868632
Epoch: 25
1.4945239648222923


In [36]:
X_test_tensor

tensor([[4.6002e-01, 4.4634e-01, 4.6528e-01, 4.7335e-01, 4.8544e-01, 4.6859e-01,
         4.6761e-01, 4.6402e-01, 4.6316e-01, 4.5157e-01, 4.7084e-01, 4.5752e-01,
         4.6378e-01, 4.6593e-01, 4.7054e-01, 4.4563e-01, 4.4899e-01, 3.9565e-01,
         3.4950e-01, 4.4632e-01, 4.4378e-01, 4.5213e-01, 6.4206e-06, 5.6519e-01],
        [4.6002e-01, 4.6002e-01, 4.4634e-01, 4.6528e-01, 4.7335e-01, 4.8544e-01,
         4.6859e-01, 4.6761e-01, 4.6402e-01, 4.6316e-01, 4.5157e-01, 4.7084e-01,
         4.5752e-01, 4.6378e-01, 4.6593e-01, 4.7054e-01, 4.4563e-01, 4.4899e-01,
         3.9565e-01, 4.4632e-01, 4.4378e-01, 4.5213e-01, 6.4206e-06, 5.6519e-01],
        [4.6002e-01, 4.6002e-01, 4.6002e-01, 4.4634e-01, 4.6528e-01, 4.7335e-01,
         4.8544e-01, 4.6859e-01, 4.6761e-01, 4.6402e-01, 4.6316e-01, 4.5157e-01,
         4.7084e-01, 4.5752e-01, 4.6378e-01, 4.6593e-01, 4.7054e-01, 4.4563e-01,
         4.4899e-01, 4.4632e-01, 4.4378e-01, 4.5213e-01, 6.4206e-06, 5.6519e-01],
        [4.6002e-01, 4.60

In [37]:
one = X_test_tensor[0]
two = X_test_tensor[1]
three = X_test_tensor[2]
four = X_test_tensor[3]
five = X_test_tensor[4]
six = X_test_tensor[5]
seven = X_test_tensor[6]


In [38]:
two[0] = 0

three[0:2] = 0 
four[0:3] = 0 
five[0:4] = 0
six[0:5] = 0 
seven[0:6] = 0 


In [39]:


one = one.to(device)
one_output = model(one)
one_output



tensor([0.4075], grad_fn=<AddBackward0>)

In [40]:
two[0] = one_output
two = two.to(device)
two_output = model(two)
two_output


tensor([0.3950], grad_fn=<AddBackward0>)

In [41]:
two[0] = one_output
two = two.to(device)
two_output = model(two)
two_output


tensor([0.3950], grad_fn=<AddBackward0>)

In [42]:
three[1] = one_output
three[0] = two_output
three = three.to(device)
three_output = model(three)
three_output

tensor([0.3846], grad_fn=<AddBackward0>)

In [43]:
four[2] = one_output
four[1] = two_output
four[0] = three_output
four = four.to(device)
four_output = model(four)
four_output

tensor([0.3775], grad_fn=<AddBackward0>)

In [44]:
five[3] = one_output
five[2] = two_output
five[1] = three_output
five[0] = four_output
five = five.to(device)
five_output = model(five)
five_output

tensor([0.3727], grad_fn=<AddBackward0>)

In [45]:
six[4] = one_output
six[3] = two_output
six[2] = three_output
six[1] = four_output
six[0] = five_output
six = six.to(device)
six_output = model(six)
six_output

tensor([0.3659], grad_fn=<AddBackward0>)

In [46]:
seven[5] = one_output
seven[4] = two_output
seven[3] = three_output
seven[2] = four_output
seven[1] = five_output
seven[0] = six_output
seven = seven.to(device)
seven_output = model(seven)
seven_output

tensor([0.3617], grad_fn=<AddBackward0>)

In [47]:
answr = [ one_output , two_output , three_output , four_output , five_output , six_output , seven_output ]
answr_total = []
for i in answr :
    i = i.detach().cpu().numpy()
    i = list(i)
    answr_total.append(i)

answr = answr_total
answr
    

[[0.40746042],
 [0.39496878],
 [0.38459808],
 [0.3774806],
 [0.37268546],
 [0.36586452],
 [0.36168203]]

In [48]:
total_preds = []
for i in answr :
    

    for j in range(24) :
        i.append(0)
    total_preds.append(i)

total_preds

[[0.40746042,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0.39496878,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0.38459808,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0.3774806,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0.37268546,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0.36586452,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0.36168203,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [49]:
preds = sc.inverse_transform(total_preds)
preds

array([[ 2.86816124e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57766000e+04,
         1.62746000e+04,  1.55042000e+04,  1.77200000e+04,
        -1.56300000e+01],
       [ 2.82859671e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57762000e+04,
         1.57762000e+04,  1.57762000e+04,  1.57766000e+04,
         1.62746000e+04,  1.55042000e+04,  1.77200000e+04,
        -1.56300000e+01],
    

In [50]:
total_preds = [i[0] for i in preds]
total_preds

[28681.612432503698,
 28285.967108511923,
 27957.49795241356,
 27732.067430615425,
 27580.19210832119,
 27364.15362033844,
 27231.682523083688]

In [51]:
dates = ["2023-07-08" , 

"2023-07-09" ,

"2023-07-10",

"2023-07-11",

"2023-07-12",

"2023-07-13",

"2023-07-14"]

In [52]:
total_preds = np.array(total_preds)
total_preds.shape

(7,)

In [53]:
result = pd.DataFrame({"Date" : dates , "Price" : total_preds})
result

,Date,Price
0,2023-07-08,28681.612433
1,2023-07-09,28285.967109
2,2023-07-10,27957.497952
3,2023-07-11,27732.067431
4,2023-07-12,27580.192108
5,2023-07-13,27364.153620
6,2023-07-14,27231.682523


In [54]:
#result.to_csv("result4_ivanborrow_poshagam.csv", index=False)